# Actividad Gaussian Mixture Model

Como se habrán dado cuenta, distintos modelos pueden modelar mejor un set de datos que otro. En esta actividad vamos a ver distintos set de datos y tratar de determinar cual de los 4 modelos (k-means, DBSCAN, hierarchical, Gaussian) podría modelar de mejor manera los datos.

In [ ]:
#Instalando las librerias requeridas
#import sys
#!{sys.executable} -m pip install numpy, pandas, plotnine, sklearn

In [ ]:
import numpy as np
import pandas as pd
from plotnine import *

In [ ]:
#Dataset 1 dos lunas
allData=[]
numPuntos=1000
from sklearn.datasets import make_moons
tempData=make_moons(n_samples=numPuntos,shuffle=True, noise=0.08, random_state=0)
allData.append(tempData[0])

#Dataset 2 dos círculos
from sklearn.datasets import make_circles
tempData = make_circles(n_samples=numPuntos, factor=.5, noise=.05)
allData.append(tempData[0])

#Dataset 3 tres clusters
from sklearn.datasets import make_blobs
tempData = make_blobs(n_samples=numPuntos, random_state=8)
allData.append(tempData[0])

#Dataset 4 clusters alargados
random_state = 170
tempData = make_blobs(n_samples=numPuntos, random_state=170)
transformation = [[0.6, -0.6], [-0.4, 0.8]]
tempData = np.dot(tempData[0], transformation)
allData.append(tempData)

#Dataset 5 Cluster con distinta varianza
tempData = make_blobs(n_samples=numPuntos,cluster_std=[1.0, 2.5, 0.5],random_state=170)
allData.append(tempData[0])

In [ ]:
from matplotlib import gridspec
import matplotlib.pyplot as plt

# Creando un gráfico vacio y grilla de gráfico
numPlots=len(allData)
numModels=5
df = pd.DataFrame(columns=[""])
fig = (ggplot(df)+theme_void()).draw()
gs = gridspec.GridSpec(numModels,numPlots)

#Agregando los puntos vacios
for i in range(numPlots):
    # Create subplots using plotnine
    temp = ggplot(df)+aes(x=allData[i][:,0],y=allData[i][:,1])+geom_point()+theme_void()
    #Adding the subplot to grid
    _ = temp._draw_using_figure(fig, [fig.add_subplot(gs[0,i])])

#showing the figure
fig.set_size_inches(numPlots*10,numModels*10)
fig.show()

# Pregunta 1
## ¿Qué datasets debería modelar sin problemas k-means?
### Una vez que responda ejecute el siguiente código

In [ ]:
#Aplicando K-means
from sklearn.cluster import KMeans
from matplotlib import gridspec
import matplotlib.pyplot as plt
from IPython.display import display

numClusters=[2, 2, 3, 3, 3]

#Aplicando K-means
for i in range(len(numClusters)):
    finalModel=KMeans(n_clusters=numClusters[i])
    finalModel=finalModel.fit(allData[i])
    temp = ggplot(df)+aes(x=allData[i][:,0],y=allData[i][:,1],color=(finalModel.labels_).astype("str"))+geom_point(show_legend=False)+theme_void()
    #Adding the subplot to grid
    _ = temp._draw_using_figure(fig, [fig.add_subplot(gs[1,i])])

#mostrando la figura
fig.set_size_inches(numPlots*10,numModels*10)
display(fig)



# Pregunta 2
## ¿Qué datasets debería modelar sin problemas DBscan?
### Una vez que responda ejecute el siguiente código

In [ ]:
#DBSCAN
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from matplotlib import gridspec
import matplotlib.pyplot as plt
from IPython.display import display
numNeighboors=3
valorEPS=[0.1, 0.2, 1.0, 0.26, 0.70]

#Aplicando DBSCAN
for i in range(len(valorEPS)):
    finalModel=DBSCAN(eps=valorEPS[i], min_samples=numNeighboors)
    finalModel=finalModel.fit(allData[i])
    temp = ggplot(df)+aes(x=allData[i][:,0],y=allData[i][:,1],color=(finalModel.labels_).astype("str"))+geom_point(show_legend=False)+theme_void()
    #Adding the subplot to grid
    _ = temp._draw_using_figure(fig, [fig.add_subplot(gs[2,i])])

#mostrando la figura
fig.set_size_inches(numPlots*10,numModels*10)
display(fig)



# Pregunta 3
## ¿Qué datasets debería modelar sin problemas algún método jerarquico?
### Una vez que responda ejecute el siguiente código

In [ ]:
#Hierarchical
import scipy.cluster.hierarchy as shc
from matplotlib import gridspec
import matplotlib.pyplot as plt
from IPython.display import display
numNeighboors=3

opcion=2 #0:Single linkage, #1:complete linkage, #2:average linkage, #3: ward
metodo=['single','complete','average','ward']
corte=np.zeros((4,5))
corte[0,:]=[0.12,0.15,2.0,0.6,1.25]
corte[1,:]=[2.5,2.25,10,6,12]
corte[2,:]=[1.5,1.2,6,3,6.4]
corte[3,:]=[25,20,50,27,75]

#Entrenando el modelo
for i in range(corte.shape[1]):
    modelo = shc.linkage(allData[i], method=metodo[opcion])
    clusters=shc.fcluster(modelo,t=corte[opcion,i],criterion="distance")-1
    temp = ggplot(df)+aes(x=allData[i][:,0],y=allData[i][:,1],color=(clusters).astype("str"))+geom_point(show_legend=False)+theme_void()
    #Agregando el grafico
    _ = temp._draw_using_figure(fig, [fig.add_subplot(gs[3,i])])

# #mostrando la figura
fig.set_size_inches(numPlots*10,numModels*10)
display(fig)



# Pregunta 4
## ¿Qué datasets debería modelar sin problemas Gaussian Mixture Model (GMM)?
### Desarrolle el código y verifique si el número de clusters detectados por el modelo para los distintos datasets

In [ ]:
from sklearn.mixture import GaussianMixture
#Busque el valor de K, usando AIC O BIC
maxNum=20 #Número máximo de clusters a evaluar
numeroDataset=3 #Número del dataset a evaluar (varía entre 0 y 4)
#Data frame para guardar todos los valores calculados
allValues=pd.DataFrame(np.zeros(((maxNum-2)*2,3)), columns=["numK","Metric","Value"])

#Ciclo para modelar distintos valores de K
for k in range(2, maxNum):
    #Cree un objeto usando GaussianMixture con k componentes y el tipo de covarianza que usted quiera
    GMM = GaussianMixture(n_components=k, covariance_type="full")

    #Entrene el modelo usando los datos del numeroDataset
    GMM = GMM.fit(allData[numeroDataset])
        
    #Calcule el BIC usando los datos de entrenamiento con la función bic
    allValues.iloc[2*(k-2),0]=k
    allValues.iloc[2*(k-2),1]= 'BIC'
    allValues.iloc[2*(k-2),2]= GMM.bic(allData[numeroDataset])   
    
    #Calcule el AIC usando los datos de entrenamiento con la función aic
    allValues.iloc[2*(k-2)+1,0]=k
    allValues.iloc[2*(k-2)+1,1]= 'AIC'
    allValues.iloc[2*(k-2)+1,2]= GMM.aic(allData[numeroDataset])
    

ggplot(allValues, aes(x='numK', y='Value'))\
+ geom_line() + theme_bw() + facet_wrap('Metric')\
+scale_x_continuous(breaks=range(2,maxNum))




In [ ]:
#Aplicando GMM
from sklearn.mixture import GaussianMixture
from matplotlib import gridspec
import matplotlib.pyplot as plt
from IPython.display import display

#Número de clusters a buscar para cada set de datos
numClusters=[2, 2, 3, 3, 3]

#Ciclo sobre el número de datasets
for i in range(len(numClusters)):
    #Aprende el modelo y genere sus predicciones
    GMM = GaussianMixture(n_components=numClusters[i], covariance_type="full")
    GMM.fit(allData[i])
    clusters= np.argmax(GMM.predict_proba(allData[i]), axis=1)


    #Guarde la predicción de los clusters en la variable clusters
    temp = ggplot(df)+aes(x=allData[i][:,0],y=allData[i][:,1],color=(clusters).astype("str"))+geom_point(show_legend=False)+theme_void()
    #Adding the subplot to grid
    _ = temp._draw_using_figure(fig, [fig.add_subplot(gs[4,i])])

#mostrando la figura
fig.set_size_inches(numPlots*10,numModels*10)
display(fig)
